<a href="https://colab.research.google.com/github/thiago2608santana/Natural_Language_Processing_with_Python/blob/main/Text_Generation_with_LSTMipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Método auxiliar

In [2]:
def read_file(filepath):
  with open(filepath) as f:
    str_text = f.read()

  return str_text

In [3]:
def separate_punc(doc_text):
  return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

**Apenas os quatro primeiros capítulos do livro Moby-Dick**

In [4]:
#read_file('/content/drive/MyDrive/NLP_with_Python_Datasets/moby_dick_four_chapters.txt')

**Livro completo Moby-Dick**

In [5]:
#read_file('/content/drive/MyDrive/NLP_with_Python_Datasets/melville-moby_dick.txt')

# Importação de bibliotecas

In [6]:
import spacy
import numpy as np
import random
from keras.preprocessing.text import Tokenizer
import tensorflow.keras as k
from tensorflow import device
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from pickle import dump, load

# Código principal

In [7]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])

**Aumentar a quantidade de palavras máximas para trabalhar com textos grandes**

In [8]:
nlp.max_length = 1198623

In [9]:
#d = read_file('/content/drive/MyDrive/NLP_with_Python_Datasets/moby_dick_four_chapters.txt')
d = read_file('/content/drive/MyDrive/NLP_with_Python_Datasets/melville-moby_dick.txt')

In [10]:
#from google.colab import drive
#drive.mount('/content/drive')

In [11]:
tokens = separate_punc(d)

/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [12]:
len(tokens)

214708

**Using #25 words --> network predict #26**

In [13]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
  seq = tokens[i-train_len:i]
  text_sequences.append(seq)

In [14]:
' '.join(text_sequences[0])

'chapter 1 loomings call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to'

In [15]:
' '.join(text_sequences[1])

'1 loomings call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest'

# Transformar as sequencias de textos em sequencias de números (tokenizer)

In [16]:
tokenizer = Tokenizer()

In [17]:
tokenizer.fit_on_texts(text_sequences)

In [18]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [19]:
sequences[0]

[158,
 9443,
 17526,
 402,
 42,
 1043,
 43,
 247,
 659,
 140,
 296,
 116,
 82,
 787,
 347,
 113,
 36,
 50,
 1788,
 6,
 49,
 3028,
 3,
 218,
 442,
 5]

**Dicionário contendo o número correspondente a cada palavra**

In [20]:
for i in sequences[0]:
  print(f'{i} : {tokenizer.index_word[i]}')
#tokenizer.index_word

158 : chapter
9443 : 1
17526 : loomings
402 : call
42 : me
1043 : ishmael
43 : some
247 : years
659 : ago
140 : never
296 : mind
116 : how
82 : long
787 : precisely
347 : having
113 : little
36 : or
50 : no
1788 : money
6 : in
49 : my
3028 : purse
3 : and
218 : nothing
442 : particular
5 : to


**Exibir a quantidade de vezes que cada palavra apareceu no texto**

In [21]:
tokenizer.word_counts

OrderedDict([('chapter', 4447),
             ('1', 28),
             ('loomings', 3),
             ('call', 1382),
             ('me', 16095),
             ('ishmael', 500),
             ('some', 15789),
             ('years', 2400),
             ('ago', 815),
             ('never', 5262),
             ('mind', 2039),
             ('how', 6330),
             ('long', 8567),
             ('precisely', 690),
             ('having', 1679),
             ('little', 6412),
             ('or', 17879),
             ('no', 14916),
             ('money', 305),
             ('in', 105799),
             ('my', 15231),
             ('purse', 178),
             ('and', 164029),
             ('nothing', 2936),
             ('particular', 1273),
             ('to', 117832),
             ('interest', 442),
             ('on', 26910),
             ('shore', 572),
             ('i', 53430),
             ('thought', 3874),
             ('would', 11232),
             ('sail', 2522),
             ('about', 

In [22]:
vocabulary_size = len(tokenizer.word_counts)

In [23]:
vocabulary_size

17526

# Transformar a lista contendo as sequências numéricas (tokens) em um array no qual cada linha representa uma sequência

In [24]:
sequences = np.array(sequences)
sequences

array([[  158,  9443, 17526, ...,   218,   442,     5],
       [ 9443, 17526,   402, ...,   442,     5,  1165],
       [17526,   402,    42, ...,     5,  1165,    42],
       ...,
       [  240,   938,   351, ...,  1419,  1313,    74],
       [  938,   351,  1418, ...,  1313,    74,   219],
       [  351,  1418,     3, ...,    74,   219,   222]])

# Separar os dados em treinamento e teste

In [25]:
sequences[:,:-1]

array([[  158,  9443, 17526, ...,     3,   218,   442],
       [ 9443, 17526,   402, ...,   218,   442,     5],
       [17526,   402,    42, ...,   442,     5,  1165],
       ...,
       [  240,   938,   351, ...,    84,  1419,  1313],
       [  938,   351,  1418, ...,  1419,  1313,    74],
       [  351,  1418,     3, ...,  1313,    74,   219]])

In [26]:
sequences[:,-1]

array([   5, 1165,   42, ...,   74,  219,  222])

In [27]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [ ]:
y = k.utils.to_categorical(y, num_classes=vocabulary_size+1)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
seq_len = X.shape[1]

# Função para criar o modelo com os parâmetros pré estabelecidos

In [ ]:
def create_model(vocabulary_size, seq_len):

  model = Sequential(name='Sequential_Text_Generator')
  model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len, name='Embedding'))
  model.add(LSTM(150, return_sequences=True, name='LSTM_1'))
  model.add(LSTM(150, name='LSTM_2'))
  model.add(Dense(150, activation='relu', name='Dense_1'))
  model.add(Dense(vocabulary_size, activation='softmax', name='Dense_2'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  model.summary()

  return model

In [ ]:
model = create_model(vocabulary_size+1, seq_len)

# Treinar o modelo

In [ ]:
with device('/device:GPU:0'):

  model.fit(X, y, batch_size=128, epochs=200)

In [ ]:
model.save('my_mobydick_model.h5')

In [ ]:
dump(tokenizer, open('my_simpletokenizer', 'wb'))

# Função que irá gerar novos textos com base no modelo treinado

In [ ]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):

    """
    Inputs:
    model : modelo já treinado no conjunto de dados
    tokenizer : tokenizer que foi ajustado no conjunto de dados
    seq_len : comprimento da sequência de treino
    seed_text : texto bruto que serve como ponto de partida do gerador
    num_gen_words : número de palavras a serem geradas pelo modelo
    """
    
    #Retorno da função
    output_text = []
    
    #Sequência de texto inicial
    input_text = seed_text
    
    #Percorrer um laço de acordo com o número de palavras geradas
    for i in range(num_gen_words):
        
        #Obter o texto de entrada e fazer um encode para uma sequência numérica
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        #Preencher as sequências para o tamanho original de treinamento
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        #Predizer a probabilidade de cada palavra pertencer a uma das classes nas quais o modelo foi treinado
        pred_word_ind = model.predict(pad_encoded, verbose=0)[0]
        index = np.argmax(pred_word_ind[0])+1
        
        #Obter uma palavra predita no vocabulário inicial
        pred_word = tokenizer.index_word[index]
        
        #Atualizar a sequência de texto de entrada, deslizando uma a uma a medida em que novas palavras são geradas
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    #Formatar o texto para uma saída mais agradável
    return ' '.join(output_text)

# Gerar texto

In [ ]:
text_sequences[0]

# Pegar uma sequência de texto aleatória dentro do conjunto de dados

In [ ]:
random.seed(101)
random_pick = random.randint(0, len(text_sequences))

In [ ]:
random_seed_text = text_sequences[random_pick]

In [ ]:
random_seed_text

In [ ]:
seed_text = ' '.join(random_seed_text)
seed_text

In [ ]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=25)